# Active learning with `superintendent`

## What is active learning?

[Active learning](https://en.wikipedia.org/wiki/Active_learning_(machine_learning)) is a
semi-supervised machine learning approach that involves labelling data to optimally train
a machine learning model.

This means a human and a machine learning algorithm interact, with the human labelling
cases the machine learning algorithm is most unsure about.

You label a few cases, train your model, make predictions about the unlabelled data, and
then label the points for which your model is not (yet) producing high-probability
predictions.

This approach is generally more efficient than labelling data points at randon, and it
allows you to reach a better model performance faster.


## Active learning in `superintendent`

The active learning process in superintendent is easy. `superintendent` is designed to work
with any machine learning model that follows the [scikit-learn](https://scikit-learn.org/stable/)
interface (i.e. the model implements a `fit` and `predict_proba` method).

You then simply pass the model, as well as a method of re-ordering the data, to a `superintendent`
widget. This then gives you a button that allows you to re-train a model.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from superintendent import SemiSupervisor

digits = load_digits()

data_labeller = SemiSupervisor.from_images(
    digits.data[:500, :],
    classifier=LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=5000),
    options=range(10),
    reorder='entropy',
)

data_labeller

Whenever you re-train a model, if you have also specified the `reorder` keyword argument, the data will be automatically re-ordered in a way prioritise the optimal subsequent data points.

Additionally, the widget will display your accuracy on the data you have already labelled. This is evaluated as the mean model score across three folds of cross-validated evaluation.

## Active learning strategies

Superintendet currently implements the following strategies:

You can implement your own strategy: the functions should simply take in a numpy array (shape *n_samples, n_classes*) of probabilities of each class for each sample, and return a ranking of the rows of that array.

For example, if sorting by margin, an input of:

| 0    	| 1    	| 2    	|
|------	|------	|------	|
| 0.1  	| 0.8  	| 0.1  	|
| 0.3  	| 0.3  	| 0.4  	|
| 0.33 	| 0.33 	| 0.34 	|
| 0.01 	| 0.01 	| 0.98 	|

Should produce an output of:

```
[2, 1, 0, 3]
```

because the third entry has the lowest margin, then the second entry, then the first, and then the last.

## Active learning for multi-output widgets

When you pass a model into a multi-labelling widget, `superintendent` will wrap your model in a 
[MultiOutputClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html)
wrapper class.

The active learning strategy will average the metric used for prioritisation (e.g. certainty,
margin) across the different classes.

## Preprocessing data before passing it to the model

In general, you will often want to pass different parts of your data to your display function
and your model. In general, superintendent does not provide "pre-model" hooks. Instead, any
pre-processing that is specific to your model or your display function, can be specified in
the `display_func`, or in a
[scikit-learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
object.

You can find an example of this [here](examples/preprocessing-data.ipynb)


## What model to choose

The choice of model is ultimately driven by the same factors that should drive
your model choice if you had a complete set of labelled data and wanted to build
a supervised machine learning model.